# Setup

In [3]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.8 MB/s eta 0:00:00


In [4]:
from datasets import load_dataset
from transformers import AutoTokenizer
import np

In [6]:
from huggingface_hub import login
login()

# Preprocessing

In [8]:
dataset_name = 'andres-vs/ruletaker-Att-Noneg-depth0'
train_data, validation_data, test_data = load_dataset(dataset_name, split=['train', 'validation', 'test'])

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [22]:
tokenizer_name = "EleutherAI/gpt-neo-125M"

tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

def tokenize(batch):
    return tokenizer(batch['input'], padding='max_length', truncation=True)

train_data_tokenized = train_data.map(tokenize, batched=True)
validation_data_tokenized = validation_data.map(tokenize, batched=True)
test_data_tokenized = test_data.map(tokenize, batched=True)

Map:   0%|          | 0/17546 [00:00<?, ? examples/s]

Map:   0%|          | 0/2494 [00:00<?, ? examples/s]

Map:   0%|          | 0/4960 [00:00<?, ? examples/s]

In [24]:
print(train_data_tokenized[:10])

{'input': ['[CLS]Anne is rough. Cold people are rough. If Anne is green then Anne is blue. Rough people are young. Anne is blue. If someone is rough and furry then they are blue. All young people are cold. If someone is rough and nice then they are green. Anne is young.[SEP]Anne is rough.[CLS]', '[CLS]Anne is rough. Cold people are rough. If Anne is green then Anne is blue. Rough people are young. Anne is blue. If someone is rough and furry then they are blue. All young people are cold. If someone is rough and nice then they are green. Anne is young.[SEP]Anne is not young.[CLS]', '[CLS]Anne is rough. Cold people are rough. If Anne is green then Anne is blue. Rough people are young. Anne is blue. If someone is rough and furry then they are blue. All young people are cold. If someone is rough and nice then they are green. Anne is young.[SEP]Anne is not green.[CLS]', '[CLS]Anne is rough. Cold people are rough. If Anne is green then Anne is blue. Rough people are young. Anne is blue. If so

In [37]:
train_data_tokenized = train_data_tokenized.rename_column('label', 'labels')
validation_data_tokenized = validation_data_tokenized.rename_column('label', 'labels')
test_data_tokenized = test_data_tokenized.rename_column('label', 'labels')
train_data_tokenized.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
validation_data_tokenized.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
test_data_tokenized.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

['input', 'labels', 'input_ids', 'attention_mask']
['input', 'labels', 'input_ids', 'attention_mask']
['input', 'labels', 'input_ids', 'attention_mask']


# Finetuning

In [40]:
from transformers import AutoModelForCausalLM

In [41]:
model = AutoModelForCausalLM.from_pretrained('roneneldan/TinyStories-8M')

config.json:   0%|          | 0.00/1.16k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/112M [00:00<?, ?B/s]

In [43]:
prompt = "Onve upon a time there was"
input_ids = tokenizer.encode(prompt, return_tensors="pt")
output = model.forward(train_data_tokenized['input_ids'][0])
# output_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(output)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Onve upon a time there was a little girl called Lucy. She was three years old and loved to explore. One day, she was walking in the park when she saw a big, shiny box. She was so excited and wanted to open it.

She asked her mum, "Mum, can I open the box?"

Her mum said, "No, Lucy. That box is too big for you. You can't open it."

But Lucy was determined and she kept asking her mum. Finally, her mum said, "Okay, Lucy. You can open the box, but you must be careful."

Lucy was so happy and she opened the box. Inside was a beautiful, shiny toy car. She was so excited and she hugged her mum.

Lucy was so happy that she had opened the box and found a new toy. She thanked her mum and they went home.

